<a href="https://colab.research.google.com/github/Nautyy9/intent_classification/blob/main/Intent_Classification_Transformer_HuggingFace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 53.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 55.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 116.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 34.4 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
import tensorflow as tf### models
import numpy as np### math computations
import matplotlib.pyplot as plt### plotting bar chart
import sklearn### machine learning library
import cv2## image processing
from sklearn.metrics import confusion_matrix, roc_curve### metrics
import seaborn as sns### visualizations
import datetime
import pathlib
import io
import os
import re
import string
import time
from numpy import random
import gensim.downloader as api
from PIL import Image
import tensorflow_datasets as tfds
import tensorflow_probability as tfp
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import Dense,Flatten,InputLayer,BatchNormalization,Dropout,Input,LayerNormalization
from tensorflow.keras.losses import BinaryCrossentropy,CategoricalCrossentropy, SparseCategoricalCrossentropy
from tensorflow.keras.metrics import Accuracy,TopKCategoricalAccuracy, CategoricalAccuracy, SparseCategoricalAccuracy
from tensorflow.keras.optimizers import Adam
from transformers import (TFDebertaForSequenceClassification, DebertaTokenizerFast, BertTokenizerFast,TFBertTokenizer,BertTokenizer,RobertaTokenizerFast,DataCollatorWithPadding,TFRobertaForSequenceClassification,TFBertForSequenceClassification,TFBertModel,create_optimizer)

In [ ]:
CONFIG={
    "BATCH_SIZE": 16,
    "NUM_EPOCHS":3,
}

In [ ]:
!pip install -q kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download -d bitext/training-dataset-for-chatbotsvirtual-assistants
!unzip "/content/training-dataset-for-chatbotsvirtual-assistants.zip" -d "/content/datasets"

  0% 0.00/1.16M [00:00<?, ?B/s]
100% 1.16M/1.16M [00:00<00:00, 77.8MB/s]
Archive:  /content/training-dataset-for-chatbotsvirtual-assistants.zip
  inflating: /content/datasets/20000-Utterances-Training-dataset-for-chatbots-virtual-assistant-Bitext-sample/20000-Utterances-Training-dataset-for-chatbots-virtual-assistant-Bitext-sample/20000-Utterances-Training-dataset-for-chatbots-virtual-assistant-Bitext-sample.csv  
  inflating: /content/datasets/20000-Utterances-Training-dataset-for-chatbots-virtual-assistant-Bitext-sample/20000-Utterances-Training-dataset-for-chatbots-virtual-assistant-Bitext-sample/20000-Utterances-Training-dataset-for-chatbots-virtual-assistant-Bitext-sample.xlsx  
  inflating: /content/datasets/20000-Utterances-Training-dataset-for-chatbots-virtual-assistant-Bitext-sample/20000-Utterances-Training-dataset-for-chatbots-virtual-assistant-Bitext-sample/LICENSE.txt  
  inflating: /content/datasets/20000-Utterances-Training-dataset-for-chatbots-virtual-assistant-Bitext-s

In [ ]:
dataset = load_dataset("csv",
                      data_files= "/content/datasets/20000-Utterances-Training-dataset-for-chatbots-virtual-assistant-Bitext-sample/20000-Utterances-Training-dataset-for-chatbots-virtual-assistant-Bitext-sample/20000-Utterances-Training-dataset-for-chatbots-virtual-assistant-Bitext-sample.csv" )

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['flags', 'utterance', 'category', 'intent'],
        num_rows: 21534
    })
})

In [ ]:
dataset['train']["intent"]

In [ ]:
?map

In [ ]:
intent_dict_keys = list(set(dataset['train']['intent']))

In [ ]:
# observe carefully! this is no normal one liner for list comprehension , here we are defining an object/dict which must have to have keys:value, the keys are  before the colon(:) and the values are after the colon , after the colon is what we've the short syntax

intent_dict = {intent_dict_keys[x]: x for x in range(len(intent_dict_keys))}
rev_intent_dict = {x :intent_dict_keys[x] for x in range(len(intent_dict_keys))}
# can only use for loop inside a list comprehension , can't be done same with the lambda function
# heho = (lambda  for x in range(len(intent_dict_keys)): intent_dict_keys[x] )


In [ ]:
rev_intent_dict

{0: 'place_order',
 1: 'get_refund',
 2: 'track_order',
 3: 'contact_human_agent',
 4: 'payment_issue',
 5: 'recover_password',
 6: 'change_shipping_address',
 7: 'contact_customer_service',
 8: 'check_cancellation_fee',
 9: 'check_refund_policy',
 10: 'change_order',
 11: 'delete_account',
 12: 'review',
 13: 'check_invoices',
 14: 'newsletter_subscription',
 15: 'set_up_shipping_address',
 16: 'track_refund',
 17: 'check_payment_methods',
 18: 'cancel_order',
 19: 'delivery_period',
 20: 'complaint',
 21: 'delivery_options',
 22: 'switch_account',
 23: 'create_account',
 24: 'registration_problems',
 25: 'get_invoice',
 26: 'edit_account'}

In [ ]:
print("intent", intent_dict[dataset["train"]["intent"][15000]])

intent 17


In [ ]:
# the differnce b/w extracting data in eager mode vs the graph mode is that the graph mode doesn't need to account for the upper dictionary like 'train' , 'val' or 'test' dir in the dataset , it directly barges into the value(features_dictionary)

def preprocess_dataset(dataset):
  # see we didn't extract train, val and test directory
  return {
      "utterance": dataset["utterance"],
      "intent": intent_dict[dataset["intent"]]
  }

In [ ]:
prep_dataset = dataset.map(preprocess_dataset)

Map:   0%|          | 0/21534 [00:00<?, ? examples/s]

In [ ]:
len(prep_dataset["train"])

21534

In [ ]:
prep_dataset["train"][0]

{'flags': 'BILC',
 'utterance': "I don't have an online account, what do I have to do to register?",
 'category': 'ACCOUNT',
 'intent': 23}

In [ ]:
model_id = 'microsoft/deberta-base'
tokenizer = DebertaTokenizerFast.from_pretrained(model_id)

In [ ]:
val = tokenizer(dataset["train"]["utterance"][0])
yuhu = {**val}
print(yuhu)

{'input_ids': [1, 100, 218, 75, 33, 41, 804, 1316, 6, 99, 109, 38, 33, 7, 109, 7, 5124, 116, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [ ]:
len(dataset["train"])

21534

In [ ]:
def tokenizer_dataset(dataset):
  return tokenizer(dataset["utterance"])
  # here see again since this ran in graph mode hence we don't need to extract the train dict from the dataset

In [ ]:
tokenized_dataset=prep_dataset.map(tokenizer_dataset)

Map:   0%|          | 0/21534 [00:00<?, ? examples/s]

In [ ]:
len(tokenized_dataset["train"])

21534

In [ ]:
# see we got the input_ids , token_type_ids and attention_mask in our dataset features dict
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['flags', 'utterance', 'category', 'intent', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 21534
    })
})

In [ ]:
# now we need to pad and collate , we can also truncate and pad with the tokenizer(returned value of the DeBertA) , by calling these attribute , but since we've done this before , thef now we use padCOllate
data_collator = DataCollatorWithPadding(tokenizer, return_tensors='tf')

# i've tried tp implement the the to_tf_dataset without collate_fn i.e, adding padding and truncation ->True but it throwed me error for irregular shape , maybe because i didn't specified the max_length but this can't be true , hence better to use padCOllate


In [ ]:
print(len(tokenized_dataset["train"]))

21534


In [ ]:
# since in this dataset we don't have the train and test splits hence we need to create them using the take and skip dataset
tf_dataset = tokenized_dataset["train"].to_tf_dataset(
    columns=["input_ids", "attention_mask", "intent"],
    collate_fn=data_collator,
    shuffle=True,
    batch_size = CONFIG["BATCH_SIZE"]
)

In [ ]:
tf_dataset

<_PrefetchDataset element_spec={'intent': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'input_ids': TensorSpec(shape=(None, None), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(None, None), dtype=tf.int64, name=None)}>

In [ ]:
len(tf_dataset)

1346

In [ ]:
for i in tf_dataset.take(1):
  print(i)

{'intent': <tf.Tensor: shape=(16,), dtype=int64, numpy=array([ 7, 25, 20,  9, 23,  4,  1, 20,  9,  7,  4, 23,  3,  1, 22, 10])>, 'input_ids': <tf.Tensor: shape=(16, 19), dtype=int64, numpy=
array([[    1,   100,    33,    10,   864,     6,   486, 19458,  7737,
            2,     0,     0,     0,     0,     0,     0,     0,     0,
            0],
       [    1, 37111,     6,   939,   236,     7,   216,   141,    38,
           64,  4016,    10, 23523,  1087,     2,     0,     0,     0,
            0],
       [    1,  4970,    41,  2936,   147,    38,    64,   146,    10,
         2267,  3674,     6,  2540,     2,     0,     0,     0,     0,
            0],
       [    1,   118,   236,     7,  1649,   110,   418,   124,   714,
            6,    99,   197,   939,   109,   116,     2,     0,     0,
            0],
       [    1,   118,  2489,  9399,   300,    10,  4392,     6,    99,
          109,   939,    33,     7,   109,     7,  1045,    65,   116,
            2],
       [    1, 37111

In [ ]:
def swap_positions(dataset):
  return {
      "input_ids": dataset["input_ids"],
      "attention_mask": dataset["attention_mask"]
  } , dataset["intent"]

In [ ]:
tf_dataset = tf_dataset.map(swap_positions)

In [ ]:
train_dataset = tf_dataset.take(int(0.9*int(len(tf_dataset))))
val_dataset = tf_dataset.skip(int(0.9*int(len(tf_dataset))))

In [ ]:
len(train_dataset)

1211

In [ ]:
val_dataset

<_SkipDataset element_spec=({'input_ids': TensorSpec(shape=(None, None), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(None, None), dtype=tf.int64, name=None)}, TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [ ]:
len(val_dataset)

135

# Moddeling

In [ ]:
model = TFDebertaForSequenceClassification.from_pretrained(model_id, num_labels=len(intent_dict))
model.summary()

All model checkpoint layers were used when initializing TFDebertaForSequenceClassification.

Some layers of TFDebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['cls_dropout', 'classifier', 'pooler']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "tf_deberta_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 deberta (TFDebertaMainLaye  multiple                  138601728 
 r)                                                              
                                                                 
 pooler (TFDebertaContextPo  multiple                  590592    
 oler)                                                           
                                                                 
 cls_dropout (TFDebertaStab  multiple                  0         
 leDropout)                                                      
                                                                 
 classifier (Dense)          multiple                  20763     
                                                                 
Total params: 139213083 (531.06 MB)
Trainable params: 139213083 (531.06 MB)
Non-trainable para

# Training

In [ ]:
CONFIG["BATCHES_PER_EPOCHS"] = len(tokenized_dataset["train"])//CONFIG["BATCH_SIZE"]
CONFIG["TOTAL_STEPS"] = int(CONFIG["BATCHES_PER_EPOCHS"] * CONFIG["NUM_EPOCHS"])

In [ ]:
print(CONFIG["TOTAL_STEPS"])

4035


In [ ]:
optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=CONFIG["TOTAL_STEPS"])

In [ ]:
model.compile(
    metrics=["accuracy"],
    optimizer = optimizer,
)

In [ ]:
history = model.fit(
    train_dataset,
    validation_data =val_dataset,
    epochs = CONFIG["NUM_EPOCHS"],
)

Epoch 1/3
1211/1211 [==============================] - 374s 239ms/step - loss: 0.0275 - accuracy: 0.9938 - val_loss: 0.0036 - val_accuracy: 0.9991
Epoch 2/3
1211/1211 [==============================] - 272s 224ms/step - loss: 0.0127 - accuracy: 0.9976 - val_loss: 0.0028 - val_accuracy: 0.9995
Epoch 3/3
1211/1211 [==============================] - 269s 222ms/step - loss: 0.0024 - accuracy: 0.9994 - val_loss: 2.6449e-04 - val_accuracy: 1.0000


In [ ]:
inputs = tokenizer(["i want to open an account!", "where i can pay my rest of the bills"],padding=True, truncation=True, return_tensors="tf")
logits = model(input_ids = inputs["input_ids"], attention_mask = inputs["attention_mask"], token_type_ids=inputs["token_type_ids"]).logits
test_value = tf.argmax(logits, axis=-1).numpy()

In [ ]:
for i in range(len(test_value)):
  print(rev_intent_dict[test_value[i]])

NameError: ignored